In [1]:
import sys
sys.path.append('..')

import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import torch
import time
import pandas as pd
from tqdm.auto import tqdm

# 导入我们自己编写的核心模块
from src.models.model_loader import load_main_llm
from src.utils.data_loader import load_sst2_dataset
from src.pipeline import AhpPipeline
from src.utils.metrics import calculate_accuracy

# 确保PyTorch能使用GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"当前使用的设备: {device}")

当前使用的设备: cuda


In [2]:
# 根据方案设置固定参数
M_CANDIDATES = 10  # 候选生成数量
K_PRUNED = 3       # 快速剪枝后保留数量
DATASET_NAME = "SST-2"
NUM_SAMPLES_TO_TEST = 100 # 为了快速得到结果，我们先在100个样本上测试

# 定义我们要对比的剪枝器列表
PRUNER_TYPES = ['semantic', 'perplexity', 'nli', 'clustering']

In [3]:
# 由于模型被下载到了/root/autodl-tmp/，我们需要使用这个“绝对路径”来加载它
# local_model_path = "/root/autodl-tmp/vicuna-7b-v1.5" 
local_model_path = "/root/autodl-tmp/circulus_alpaca-7b"

print(f"准备从本地路径加载主模型: {local_model_path}")

# 加载主LLM
# use_4bit=True 选项会使用4-bit量化，极大节省显存
main_model, main_tokenizer = load_main_llm(model_name=local_model_path, use_4bit=True)

# 加载SST-2测试集
# 第一次运行时会自动从Hugging Face Hub下载
sst2_test_dataset = load_sst2_dataset(split='validation')

# 为了方便，我们将数据转换为Pandas DataFrame，并只取前N个样本
sst2_test_df = sst2_test_dataset.to_pandas().head(NUM_SAMPLES_TO_TEST)
print(f"\n成功加载SST-2测试集的前 {NUM_SAMPLES_TO_TEST} 条数据。")
sst2_test_df.head()

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


准备从本地路径加载主模型: /root/autodl-tmp/circulus_alpaca-7b
正在加载主模型: /root/autodl-tmp/circulus_alpaca-7b...


The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

主模型加载成功。
正在加载SST-2数据集 (validation split)...


'(MaxRetryError("HTTPSConnectionPool(host='hf-mirror.com', port=443): Max retries exceeded with url: /datasets/glue/resolve/main/README.md (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fc9e46ecc20>, 'Connection to hf-mirror.com timed out. (connect timeout=10)'))"), '(Request ID: 4cd80d44-e014-4e47-afc9-f46af0f3e098)')' thrown while requesting HEAD https://hf-mirror.com/datasets/glue/resolve/main/README.md
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='hf-mirror.com', port=443): Read timed out. (read timeout=10)"), '(Request ID: 61cf11a6-489d-4f6b-90c9-3b1d6fb2fff1)')' thrown while requesting HEAD https://hf-mirror.com/api/resolve-cache/datasets/nyu-mll/glue/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c/README.md
Retrying in 1s [Retry 1/5].


SST-2数据集加载成功。

成功加载SST-2测试集的前 100 条数据。


,sentence,label,idx,label_text
0,it 's a charming and often affecting journey .,1,0,positive
1,unflinchingly bleak and desperate,0,1,negative
2,allows us to hope that nolan is poised to emba...,1,2,positive
3,"the acting , costumes , music , cinematography...",1,3,positive
4,"it 's slow -- very , very slow .",0,4,negative


In [4]:
# # (单元格 4 - 新版调试专用)

# pruner_name_to_debug = 'nli'
# pipeline = AhpPipeline(
#     main_model=main_model,
#     main_tokenizer=main_tokenizer,
#     pruner_name=pruner_name_to_debug,
#     k_val=K_PRUNED,
#     m_val=M_CANDIDATES
# )

# sample_sentence = sst2_test_df.iloc[0]['sentence']
# sample_label = sst2_test_df.iloc[0]['label_text']

# print(f"正在调试样本: '{sample_sentence}'")
# print(f"真实标签: '{sample_label}'")

# # (修改点) 不再传入 sample_label
# prediction = pipeline.predict_single(sample_sentence)

# print(f"\n最终聚合预测结果: '{prediction}'")

正在加载NLI模型: roberta-large-mnli...
NLI模型加载成功。
正在调试样本: 'it 's a charming and often affecting journey . '
真实标签: 'positive'


Calculating NLI Scores:   0%|          | 0/10 [00:00<?, ?it/s]


--- [DEBUG] Final Prediction Step ---
Input Prompt:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Analyze the sentiment of the following movie review. Respond with a single word: 'positive' or 'negative'.

Review: "It's a charming and often affecting journey."

### Response:

Raw Prediction Text: 'positive'
Cleaned Prediction: 'positive'
-------------------------------------


最终聚合预测结果: 'positive'


In [5]:
# (单元格 4 - 恢复为正式实验代码)

results = []

# 遍历每一种剪枝方法
for pruner_name in PRUNER_TYPES:
    print(f"\n{'='*20} 正在测试剪枝器: {pruner_name.upper()} {'='*20}")

    pipeline = AhpPipeline(
        main_model=main_model,
        main_tokenizer=main_tokenizer,
        pruner_name=pruner_name,
        k_val=K_PRUNED,
        m_val=M_CANDIDATES
    )

    predictions = []
    start_time = time.time()

    # 遍历测试数据集
    for index, row in tqdm(sst2_test_df.iterrows(), total=len(sst2_test_df), desc=f"Processing with {pruner_name}"):
        sentence = row['sentence']

        # (注意) 不再需要传入真实标签
        prediction = pipeline.predict_single(sentence)
        predictions.append(prediction)

    end_time = time.time()

    # 计算指标
    total_time = end_time - start_time
    avg_time_per_sample = total_time / NUM_SAMPLES_TO_TEST

    accuracy = calculate_accuracy(sst2_test_df['label_text'].tolist(), predictions)

    # 保存结果
    results.append({
        "筛选规则 (Pruning Rule)": pruner_name,
        "任务准确率 (Task Accuracy)": accuracy,
        "计算开销 (s/sample)": avg_time_per_sample
    })

    print(f"剪枝器 '{pruner_name}' 完成测试。")
    print(f" -> 准确率: {accuracy:.4f}")
    print(f" -> 平均耗时: {avg_time_per_sample:.4f} 秒/样本")


==================== 正在测试剪枝器: SEMANTIC ====================
正在加载剪枝模型: sentence-transformers/all-mpnet-base-v2...
剪枝模型加载成功。


Processing with semantic:   0%|          | 0/100 [00:00<?, ?it/s]


--- [DEBUG] Final Prediction Step ---
Input Prompt:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Analyze the sentiment of the following movie review. Respond with a single word: 'positive' or 'negative'.

Review: "It's a charming and often affecting journey ."

### Response:

Raw Prediction Text: 'positive'
Cleaned Prediction: 'positive'
-------------------------------------


--- [DEBUG] Final Prediction Step ---
Input Prompt:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Analyze the sentiment of the following movie review. Respond with a single word: 'positive' or 'negative'.

Review: "The situation was unflinchingly bleak and desperate."

### Response:

Raw Prediction Text: 'Negative'
Cleaned Prediction: 'negative'
-------------------------------------


--- [DEBUG] Final Prediction Step ---
Input Prompt:
Below is an instructio

Processing with perplexity:   0%|          | 0/100 [00:00<?, ?it/s]

Calculating PPL:   0%|          | 0/10 [00:00<?, ?it/s]

...


--- [DEBUG] Final Prediction Step ---
Input Prompt:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Analyze the sentiment of the following movie review. Respond with a single word: 'positive' or 'negative'.

Review: "Why make a documentary about these marginal historical figures?"

### Response:

Raw Prediction Text: 'Negative'
Cleaned Prediction: 'negative'
-------------------------------------

剪枝器 'nli' 完成测试。
 -> 准确率: 0.9600
 -> 平均耗时: 2.5565 秒/样本

==================== 正在测试剪枝器: CLUSTERING ====================
正在加载剪枝模型: sentence-transformers/all-mpnet-base-v2...


'(ReadTimeoutError("HTTPSConnectionPool(host='hf-mirror.com', port=443): Read timed out. (read timeout=10)"), '(Request ID: 9750374f-409e-4ac0-984e-bfbaab2c70b5)')' thrown while requesting HEAD https://hf-mirror.com/sentence-transformers/all-mpnet-base-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


剪枝模型加载成功。


Processing with clustering:   0%|          | 0/100 [00:00<?, ?it/s]


--- [DEBUG] Final Prediction Step ---
Input Prompt:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Analyze the sentiment of the following movie review. Respond with a single word: 'positive' or 'negative'.

Review: "It is a charming and often affecting journey."

### Response:

Raw Prediction Text: 'positive'
Cleaned Prediction: 'positive'
-------------------------------------


--- [DEBUG] Final Prediction Step ---
Input Prompt:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Analyze the sentiment of the following movie review. Respond with a single word: 'positive' or 'negative'.

Review: "The situation became unflinchingly bleak and desperate."

### Response:

Raw Prediction Text: 'Negative'
Cleaned Prediction: 'negative'
-------------------------------------


--- [DEBUG] Final Prediction Step ---
Input Prompt:
Below is an instruc

In [6]:
# 将结果转换为DataFrame
results_df = pd.DataFrame(results)

print("\n实验一：最优筛选规则确定 - 结果汇总")
print("=" * 60)
print(f"数据集: {DATASET_NAME}, 测试样本数: {NUM_SAMPLES_TO_TEST}, M={M_CANDIDATES}, K={K_PRUNED}")
print("-" * 60)

# 按照准确率降序排序
results_df_sorted = results_df.sort_values(by="任务准确率 (Task Accuracy)", ascending=False)
print(results_df_sorted.to_string(index=False))


实验一：最优筛选规则确定 - 结果汇总
数据集: SST-2, 测试样本数: 100, M=10, K=3
------------------------------------------------------------
筛选规则 (Pruning Rule)  任务准确率 (Task Accuracy)  计算开销 (s/sample)
                nli                   0.96         2.556520
         clustering                   0.93         2.497312
         perplexity                   0.92         2.542323
           semantic                   0.92         2.502659


In [7]:
# (修改后) - 保存SST-2实验结果，并自动编号以防覆盖

# 确保我们的results目录存在
if not os.path.exists('../results'):
    os.makedirs('../results')
    print("创建 'results' 文件夹。")

# --- 新增逻辑：生成不会被覆盖的文件名 ---
# 1. 定义文件名的基础部分和扩展名
base_path = '../results/experiment_1_sst2_results'
extension = '.csv'
save_path_sst2 = f"{base_path}{extension}"

# 2. 检查文件是否存在，如果存在，则在文件名后附加一个数字
counter = 1
while os.path.exists(save_path_sst2):
    # 例如，生成 "experiment_1_sst2_results_1.csv"
    save_path_sst2 = f"{base_path}_{counter}{extension}"
    counter += 1
# --- 逻辑结束 ---


# 使用 to_csv 方法保存DataFrame
# index=False 表示我们不把DataFrame的行索引写入到文件中
results_df_sorted.to_csv(save_path_sst2, index=False)

print(f"\n实验结果已成功保存到: {save_path_sst2}")


实验结果已成功保存到: ../results/experiment_1_sst2_results_1.csv
